In [1]:
!unzip '/content/open.zip'

Archive:  /content/open.zip
   creating: open/
  inflating: open/sample_submission.csv  
  inflating: open/test.csv           
  inflating: open/train.csv          


In [21]:
!pip install -U mljar-supervised
from supervised.automl import AutoML

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.1/128.1 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.9/547.9 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.2 MB/s eta 0:00:00
  Created wheel for mljar-supervised: filename=mljar_supervised-1.0.2-py3-none-any.whl size=163289 sha256=1003c9fbba7ef5553422fa956051871b4d9f460ca78003ccba06c20dd2ff3914
  Stored in directory: /root/.cache/pip/wheels/4b/cb/0a/386e3ab

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [8]:
import pandas as pd
import numpy as np
import random
import os

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

train = pd.read_csv('./open/train.csv')
test = pd.read_csv('./open/test.csv')

In [10]:
# train_x는 독립변수이므로 종속변수(풍속 (m/s))를 제거합니다.
# 또한 target 이외의 분석에 활용하지 않는 데이터(id)를 제거합니다.
x_train = train.drop(columns=['ID', '풍속 (m/s)','측정 시간대','월',	'일'], axis = 1)

# train_y는 종속변수로 값을 설정합니다.
y_train = train['풍속 (m/s)']
# train에서와 마찬가지로 분석에 활용하지 않는 데이터(id)를 제거합니다.
x_test = test.drop(columns=['ID','측정 시간대','월',	'일'])

In [16]:
x_train.columns

Index(['섭씨 온도(°⁣C)', '절대 온도(K)', '이슬점 온도(°C)', '상대 습도 (%)', '대기압(mbar)',
       '포화 증기압(mbar)', '실제 증기압(mbar)', '증기압 부족량(mbar)', '수증기 함량 (g/kg)',
       '공기 밀도 (g/m**3)', '풍향 (deg)'],
      dtype='object')

전처리

In [18]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
ss = StandardScaler().fit(x_train)
x_train_scaled = ss.transform(x_train)
x_test_scaled = ss.transform(x_test)

x_train_scaled = pd.DataFrame(x_train_scaled,columns=x_train.columns)
x_test_scaled = pd.DataFrame(x_test_scaled,columns=x_test.columns)

In [19]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le = le.fit(train['측정 시간대'])
x_train_scaled['측정 시간대'] = le.transform(train['측정 시간대'])
x_test_scaled['측정 시간대'] = le.transform(test['측정 시간대'])
x_train_scaled['월'] = train['월']; x_train_scaled['일'] = train['일']
x_test_scaled['월'] = test['월']; x_test_scaled['일'] = test['일']

In [22]:
x_train_scaled.head()

,섭씨 온도(°⁣C),절대 온도(K),이슬점 온도(°C),상대 습도 (%),대기압(mbar),포화 증기압(mbar),실제 증기압(mbar),증기압 부족량(mbar),수증기 함량 (g/kg),공기 밀도 (g/m**3),풍향 (deg),측정 시간대,월,일
0,0.482855,0.455931,0.693376,0.016942,0.231977,0.240582,0.566963,-0.082878,0.558184,-0.417949,-0.248661,3,7,2
1,0.843420,0.823415,1.056902,-0.157090,0.117521,0.655413,1.047176,0.175258,1.040291,-0.781995,0.004949,1,8,21
2,-0.028250,-0.015692,-0.008387,-0.101151,-0.149921,-0.239880,-0.192777,-0.217725,-0.191340,-0.034423,-0.360060,3,11,1
3,-0.573347,-0.798540,-0.433554,0.476885,2.744341,-0.637375,-0.563090,-0.535580,-0.605650,1.287678,1.041906,1,12,28
4,0.893195,0.824612,1.142251,-0.113582,0.650138,0.718566,1.169021,0.175258,1.145751,-0.687631,-1.863469,2,9,26


In [ ]:
# automl 모델 생성
automl = AutoML(mode="Optuna", eval_metric='mae', ml_task='regression', n_jobs=-1, optuna_time_budget=1800, total_time_limit=24*3600)
automl.fit(x_train_scaled, y_train)
automl.save('/content/drive/MyDrive/automl_wind')

In [26]:
automl.report()

In [ ]:
# {'learning_rate': 0.05, 'num_leaves': 1930, 'lambda_l1': 1.5981003676194843e-06, 'lambda_l2': 0.00014578096765781285, 'feature_fraction': 0.6181783253445614, 'bagging_fraction': 0.9976637360927889, 'bagging_freq': 3, 'min_data_in_leaf': 7, 'extra_trees': False}. Best is trial 43 with value: 0.29772083056019083.

In [ ]:
automl = AutoML(
    algorithms=[ "CatBoost"],
    mode="Optuna",
    optuna_time_budget=1200,
    total_time_limit=24*3600
)
automl.fit(x_train_scaled, y_train)
automl.save('/content/drive/MyDrive/automl_wind')

AutoML directory: AutoML_3
Expected computing time:
Time for tuning with Optuna: len(algorithms) * optuna_time_budget = 1200 seconds
There is no time limit for ML model training after Optuna tuning (total_time_limit parameter is ignored).
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['CatBoost']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble', 'stack', 'ensemble_stacked']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 1 model


[I 2023-07-19 08:49:37,722] A new study created in memory with name: no-name-5b4be10e-c263-4fb6-8b92-e26c2d41c867


Optuna optimizes CatBoost with time budget 1200 seconds eval_metric rmse (minimize)


[I 2023-07-19 08:49:52,703] Trial 0 finished with value: 0.4563215685207894 and parameters: {'learning_rate': 0.1, 'depth': 8, 'l2_leaf_reg': 7.7997800836072235, 'random_strength': 2.7259260601004898, 'rsm': 0.34881782962878705, 'min_data_in_leaf': 81}. Best is trial 0 with value: 0.4563215685207894.
[I 2023-07-19 08:49:59,342] Trial 1 finished with value: 0.5409314669093543 and parameters: {'learning_rate': 0.05, 'depth': 6, 'l2_leaf_reg': 6.834661005427845, 'random_strength': 7.127020272701981, 'rsm': 0.43322567931135547, 'min_data_in_leaf': 57}. Best is trial 0 with value: 0.4563215685207894.
[I 2023-07-19 08:50:10,882] Trial 2 finished with value: 0.43828890388637637 and parameters: {'learning_rate': 0.2, 'depth': 9, 'l2_leaf_reg': 3.648923350415333, 'random_strength': 6.1539617881809745, 'rsm': 0.16784311747867892, 'min_data_in_leaf': 37}. Best is trial 2 with value: 0.43828890388637637.
[I 2023-07-19 08:50:29,982] Trial 3 finished with value: 0.4798967708819802 and parameters: {'

참고

https://levelup.gitconnected.com/10-python-libraries-for-automated-machine-learning-that-you-should-think-to-use-in-2023-730a2694f221